# Logistic Regression Consulting Project

This is a project I completed as part of Jose Portilla's Udemy course on [Spark and Python for Big Data with Pyspark](https://www.udemy.com/course/spark-and-python-for-big-data-with-pyspark/). This project aims to demonstrate:

- My understanding of logistic regression
- My ability to use PySpark to clean and preprocess data
- My ability to make accurate predictions based on business data

## Project Outline
### Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads `Purchased`
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('churn').getOrCreate()
df = spark.read.csv('dbfs:/FileStore/tables/customer_churn.csv',inferSchema=True,header=True)
holdout = spark.read.csv('dbfs:/FileStore/tables/new_customers.csv',inferSchema=True,header=True)

What are the relevant variables going to be? Age, Total_Purchase, Years, Num_sites, Onboard_date, Location all sound relevant. Account_Manager is currently randomly assigned, so it's not going to be useful. 

We need to convert Onboard_date to a number of days since the client was onboarded as well.

In [0]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [0]:
df.createOrReplaceTempView('churn')
spark.sql('SELECT * FROM churn').show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [0]:
spark.sql(
    'SELECT * FROM churn WHERE Names IS NULL OR Age IS NULL'
    ).show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+



In [0]:
df_cleaned = df.na.drop()

In [0]:
df_cleaned.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

From this we can see there are no missing values, and we can proceed with further pre-processing of the data. 
Our next tasks are the following:
- Indexing and one-hot encoding of Location
- Processing of Onboard_date

Let's start by processing the Onboard_date. I'm going to use `DATEDIFF()` from SQL to determine the number of days since the client was onboarded.

In [0]:
df_cleaned.createOrReplaceTempView('df_cleaned')
df_processed = spark.sql(
    "SELECT *, datediff(GETDATE(),Onboard_date) AS Days_since_onboarded FROM df_cleaned"
)
df_processed.head(1)
# Footnote: having trained on SQL Server, I was unused to DATEDIFF() only taking two arguments - the interval defaults to days though

Out[25]: [Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, Days_since_onboarded=4199)]

Now to index and encode Location. We aren't going to use the objects yet as they will go into the Pipeline later on.

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
loc_indexer = StringIndexer(inputCol='Location', outputCol='LocIndex',handleInvalid="keep")
# I turned on "handleInvalid" to account for unseen locations in the test data. In retrospect, Location may not be a very useful
# feature as the locations actually seem to be addresses - I was thinking they would be e.g. city names
loc_encoder = OneHotEncoder(inputCol='LocIndex', outputCol='LocVec')

With all the features in the appropriate format, we can now vectorise the appropriate columns for use in the logistic regression.

In [0]:
# Imports
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [0]:
# Show the columns that we need
df_processed.columns

Out[56]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'Days_since_onboarded']

In [0]:
assembler = VectorAssembler(inputCols=[
    'Age','Total_Purchase','Years','Num_Sites','LocVec','Days_since_onboarded'
    ],outputCol='features')
log_reg_churn = LogisticRegression(featuresCol='features',labelCol='Churn')

pipeline = Pipeline(stages=[loc_indexer, loc_encoder,assembler,log_reg_churn])

In [0]:
# Split into test and train, run the pipeline on the training data and then run the model on the test data
train_data, test_data = df_processed.randomSplit([0.7,0.3])
fit_model = pipeline.fit(train_data)
results = fit_model.transform(test_data)

Now it's time to evaluate the data by looking at the area under the ROC curve.

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')


In [0]:
results.select("Churn", "prediction").show(5)

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
+-----+----------+
only showing top 5 rows



In [0]:
AUC = my_eval.evaluate(results)
AUC

Out[80]: 0.5

Well, that's not what we want! I think I know what the problem is - the Location feature is going to be different for every place, and the model massively overfits as the location can be used to perfectly predict the outcome. I'm going to run it all again, and remove the location feature.

In [0]:
# Not including location or the location indexing/encoding this time
assembler2 = VectorAssembler(inputCols=[
    'Age','Total_Purchase','Years','Num_Sites','Days_since_onboarded'
    ],outputCol='features')
log_reg_churn2 = LogisticRegression(featuresCol='features',labelCol='Churn')

pipeline2 = Pipeline(stages=[assembler2,log_reg_churn2])

fit_model2 = pipeline2.fit(train_data)
results2 = fit_model2.transform(test_data)

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

AUC = my_eval.evaluate(results2)
AUC

Out[82]: 0.7202380952380951

A much better result. Let's now deploy the model by predicting on our holdout set.

In [0]:
# Start by using SparkSQL to add the days since onboarded column
holdout.createOrReplaceTempView('holdout')
holdout_processed = spark.sql(
    "SELECT *, datediff(GETDATE(),Onboard_date) AS Days_since_onboarded FROM holdout"
)


In [0]:
deployed_results = fit_model2.transform(holdout_processed)

In [0]:
deployed_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Our model has predicted that the following companies are likely to churn: Cannon-Benson, Barron-Robertson, Sexton-Golden, and Parks-Robbins. The agency ought to consider assigning an account manager to each of these four. 